## Problem 1

In [ ]:
import re

In [2]:
import os
import re
from bs4 import BeautifulSoup

# --- Cấu hình ---
DATA_DIR = "/home/temp-user/workspace/plate-recognition-low-resolution/out/agencyvietnam"
OUT_FILE = "/home/temp-user/workspace/plate-recognition-low-resolution/agency_contacts.txt"

# Regrex theo yêu cầu (Lưu ý: bỏ dấu $ ở cuối email để findall tìm được trong văn bản)
email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern = r"(?:\+84|84|0)(?:\s|\.|\-|\(|\))?(?:\d(?:\s|\.|\-|\(|\))?){9,11}"

def decode_cf_email(encoded_str):
    """Giải mã email bị Cloudflare obfuscate"""
    r = int(encoded_str[:2], 16)
    return "".join([chr(int(encoded_str[i:i+2], 16) ^ r) for i in range(2, len(encoded_str), 2)])

results = []

# Duyệt qua các file HTML đã tải về
files = [f for f in os.listdir(DATA_DIR) if f.endswith(".html") and f != "index.html"]
files.sort()

print(f"Bắt đầu xử lý {len(files)} file...")

for filename in files:
    file_path = os.path.join(DATA_DIR, filename)
    with open(file_path, "r", encoding="utf-8") as f:
        html_content = f.read()
    
    soup = BeautifulSoup(html_content, "html.parser")
    
    # 1. Lấy tên Agency (thẻ h1)
    name_el = soup.select_one("h1.display-5")
    name = name_el.get_text(strip=True) if name_el else "Unknown"
    
    # 2. Tìm danh sách Email
    emails = set(re.findall(email_pattern, html_content, re.IGNORECASE))
    # Giải mã thêm các email từ Cloudflare (nếu có)
    for cf in soup.select("[data-cfemail]"):
        try:
            emails.add(decode_cf_email(cf["data-cfemail"]))
        except: continue
        
    # 3. Tìm danh sách Số điện thoại
    phones = set(re.findall(phone_pattern, html_content))
    
    results.append({
        "name": name,
        "emails": sorted(list(emails)),
        "phones": sorted(list(phones)),
        "file": filename
    })

# --- Lưu kết quả vào file ---
with open(OUT_FILE, "w", encoding="utf-8") as f:
    for res in results:
        f.write(f"Agency: {res['name']}\n")
        f.write(f"File: {res['file']}\n")
        f.write(f"Emails: {', '.join(res['emails']) if res['emails'] else 'N/A'}\n")
        f.write(f"Phones: {', '.join(res['phones']) if res['phones'] else 'N/A'}\n")
        f.write("-" * 40 + "\n")

print(f"Xử lý xong! Kết quả đã được lưu tại: {OUT_FILE}")

Bắt đầu xử lý 10 file...
Xử lý xong! Kết quả đã được lưu tại: /home/temp-user/workspace/plate-recognition-low-resolution/agency_contacts.txt


## Problem 2

In [2]:
import os
import re
from underthesea import word_tokenize, ner, classify
from tqdm import tqdm

# --- Cấu hình ---
DATA_DIR = "/home/temp-user/workspace/plate-recognition-low-resolution/out/travel"
LOCATIONS = {
    "hanoi": ["Hà Nội", "Thủ đô", "Hoàn Kiếm", "Ba Đình"],
    "danang": ["Đà Nẵng", "Sông Hàn", "Ngũ Hành Sơn"],
    "quangbinh": ["Quảng Bình", "Sơn Đoòng", "Phong Nha", "Đồng Hới"]
}

# Khởi tạo kết quả
category_map = {} # Phân loại theo category
location_files = {loc: [] for loc in LOCATIONS} # Phân loại theo địa điểm

# Lấy danh sách file bài viết
files = [f for f in os.listdir(DATA_DIR) if f.endswith(".txt") and f != "urls.txt"]
files.sort()

# Sử dụng tqdm để theo dõi tiến độ
for filename in tqdm(files, desc="Processing Articles"):
    path = os.path.join(DATA_DIR, filename)
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        if len(lines) < 3: continue
        
        title = lines[0].strip()
        url = lines[2].strip()
        body = "".join(lines[4:]).strip()
        full_text = title + " " + body

    # 1. Word Tokenize (từ ghép nối bởi _) và NER
    tokenized_text = word_tokenize(full_text, format="fixed")
    entities = ner(full_text)
    
    # 2. Phân loại bài viết (Classify)
    try:
        cat = classify(full_text)
        cat_name = cat[0] if cat else "Khác"
    except: cat_name = "Khác"
    
    if cat_name not in category_map: category_map[cat_name] = []
    category_map[cat_name].append(f"{title} - {url}")

    # 3. Lọc theo từ khóa địa điểm
    for loc_key, keywords in LOCATIONS.items():
        if any(kw.lower() in full_text.lower() for kw in keywords):
            location_files[loc_key].append(f"{title}\n{url}\n")

# --- Lưu kết quả ---
# Lưu file địa điểm
for loc, content in location_files.items():
    if content:
        out_path = os.path.join(os.path.dirname(DATA_DIR), f"{loc}.txt")
        with open(out_path, "w", encoding="utf-8") as f:
            f.write("\n".join(content))

# Lưu file phân loại category
summary_path = os.path.join(os.path.dirname(DATA_DIR), "categories_summary.txt")
with open(summary_path, "w", encoding="utf-8") as f:
    for cat, articles in category_map.items():
        f.write(f"=== DANH MỤC: {cat.upper()} ===\n")
        f.write("\n".join(articles) + "\n\n")

print("\nXử lý hoàn tất!")
print(f"- Kết quả lưu tại thư mục: {os.path.dirname(DATA_DIR)}")

Processing Articles: 100%|██████████| 100/100 [06:33<00:00,  3.93s/it]


Xử lý hoàn tất!
- Kết quả lưu tại thư mục: /home/temp-user/workspace/plate-recognition-low-resolution/out
